# Generating Permutations and the Synthetic Fault
This notebook creates a sketch illustrating the permutations of data points
violating the $d_\mathrm{min}$ criterion.

In [ ]:
import numpy as np
from plotconfig import *
from cmcrameri.cm import *
import matplotlib.pyplot as plt
from pdtoolbox import gamma_cdf, gamma_mle
from zeal2022hf import get_cm_colors
from matplotlib.patches import Circle, Arrow
from matplotlib.collections import LineCollection

In [ ]:
rng = np.random.default_rng(25499783122)
xy = 160. * (rng.random(size=(10,2)) - 0.5)

In [ ]:
pairs = []
for i in range(xy.shape[0]):
    for j in range(i+1,xy.shape[0]):
        d = np.sqrt(((xy[i] - xy[j])**2).sum())
        if d <= 20.0:
            pairs.append((i,j))
print(pairs)

In [ ]:
colors = get_cm_colors(vik, 9)
color0 = colors[0]
color1 = colors[3]
color2 = colors[8]
color3 = colors[5]

In [ ]:
fig = plt.figure(figsize=(5.3, 1.2))
ax_bg = fig.add_axes((0,0,1,1))
ax_bg.set_axis_off()
ax_bg.add_patch(Arrow(0.18, 0.4, 0.06, 0.0, width=0.2, color='tab:gray'))
width = 0.75 * 0.22
ax = fig.add_axes((0.005, 0.01, width, 0.8))
ax.scatter(*xy.T, marker='.', edgecolor='none', color=color0)
ax.set_aspect('equal')
ax.set_title('(a)')
for i in range(xy.shape[0]):
    ax.add_patch(Circle(xy[i], 20.0, facecolor='none', edgecolor='k', linewidth=0.5))
for pair in pairs:
    ax.plot(*xy[pair,:].T, color='k', linewidth=1.0, zorder=0)
ax.set_xticks([])
ax.set_yticks([])
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# Now the four combinations:
Dy = -100.0
x = -80.0
k = 0
x0 = 0.25
dx = 0.75 * 0.25
for i in pairs[0]:
    for j in pairs[1]:
        mask = np.ones(xy.shape[0],dtype=bool)
        mask[i] = False
        mask[j] = False
        ax = fig.add_axes((x0+k*dx, 0.01, width, 0.8))
        ax.set_title(['(b)', '(c)', '(d)','(e)'][k])
        ax.set_aspect('equal')
        ax.scatter(*xy[mask,:].T, marker='.', edgecolor='none', color=color0)
        ax.scatter(*xy[~mask,:].T, color=color1, zorder=0, marker='.', edgecolor='none')
        for pair in pairs:
            ax.plot(*xy[pair,:].T, color='k', linewidth=1.0, zorder=-1)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        ax.set_xticks([])
        ax.set_yticks([])
        
        k += 1

fig.savefig('figures/A11-Sketch-Data-d_min-Permutation.pdf')

## Synthetic Fault

In [ ]:
ar = np.deg2rad(-15.0)
car = np.cos(ar)
sar = np.sin(ar)
xf = 80 * sar * np.array((1, -1))
yf = 80 * car * np.array((1, -1))
M = np.array(((car, -sar), (sar, car)))

In [ ]:
rng = np.random.default_rng(3892989)
xy = 160 * (rng.random((30,2)) - 0.5)
xy = xy[np.sum(xy**2, axis=1) < 80**2]
xy_projected = xy.copy()
xy_projected[:,0] = 0.0

In [ ]:
fig = plt.figure(figsize=(2.7,2.7))
ax = fig.add_axes((0,0,1,1))
ax.set_aspect('equal')
ax.set_axis_off()
ax.set_xlim(-100, 100)
ax.set_ylim(-100,100)
ax.plot(xf,yf, color=color0, linewidth=2)
ax.scatter(*(xy @ M).T, marker='.', edgecolor='none', facecolor=color2)
ax.add_collection(LineCollection([(xy_from, xy_to) for xy_from, xy_to in zip(xy @ M, xy_projected @ M)],
                                 color=color3, zorder=0, linewidth=0.8, linestyle='--'))
ax.add_artist(Circle((0,0), 80.0, facecolor='none', edgecolor='k', linewidth=0.8))
ax.text(0.13*xf[0]-0.09*yf[0], 0.13*yf[0]+0.09*xf[0], 'Fault trace', rotation=-90-np.rad2deg(ar),
        ha='center', va='center')
ax.text(0.36*xf[0]+0.6*yf[0], 0.36*yf[0]-0.6*xf[0], 'Distance\nto fault', rotation=-np.rad2deg(ar),
        ha='center', va='center')
ax.annotate('Data point', (xy @ M)[1], xytext=(30, 0.0),
            arrowprops = {
                'arrowstyle' : '-|>',
                'connectionstyle' : 'arc3,rad=0.2',
                'linewidth' : 0.8,
                'facecolor' : 'k'
            })
fig.savefig('figures/A11-Sketch-Synthetic-Fault.pdf')

## The Illustration of the Bayesian Anomaly Quantification

In [ ]:
alpha = 180.0
beta = alpha / 68.3
N = 20
d = 18e3
P_ano = 140e6
Q = 2 * P_ano / (d * 160e3)

In [ ]:
rng = np.random.default_rng(291983)
q = rng.gamma(alpha, size=N) / beta
x = 160e3 * (rng.random(N) - 0.5)

In [ ]:
def anomaly(x):
    x_ = np.atleast_1d(x)
    z = np.zeros_like(x_)
    mask = x != 0
    z[mask] = Q / np.pi * (1.0 - x_[mask]/d * np.arctan(d/x_[mask]))
    z[~mask] = Q / np.pi
    if x_.size == 1:
        return float(z)
    return z

In [ ]:
dq = 1e3 * anomaly(x)

In [ ]:
from reheatfunq import HeatFlowAnomalyPosterior
from reheatfunq.anomaly import AnomalyLS1980
from reheatfunq import GammaConjugatePrior

In [ ]:
gcp = GammaConjugatePrior(1.0, 0.0, 0.0, 0.0)

In [ ]:
ano = AnomalyLS1980(np.array([(0.0, 80e3), (0.0, -80e3)]), d)
hfap = HeatFlowAnomalyPosterior(q+dq, x, np.zeros_like(x), ano, gcp, dmin=0.0)
P_H = np.linspace(0.0, hfap.PHmax, 200)
y = hfap.pdf(P_H)

In [ ]:
fig = plt.figure(figsize=(6.3,1.7))
#ax_bg = fig.add_axes((0,0,1,1))
ax0 = fig.add_axes((0.058, 0.2, 0.24, 0.77))
h0_0 = ax0.scatter(1e-3*x, q, marker='v', edgecolor='none', color=color1)
l0 = 'Heat flow\n(undisturbed)'
h1_0 = ax0.scatter(1e-3*x, q+dq, color=color0, edgecolor='none', s=20)
l1 = 'Heat flow\nwith anomaly'
ax0.add_collection(LineCollection([((1e-3*x, q0), (1e-3*x, q1)) for x,q0,q1 in zip(x, q, q+dq)],
                                  color=color1, linewidth=1.0, zorder=0, linestyle=':'))
twax = ax0.twinx()
xplot = np.linspace(-80e3, 80e3, 101)
ax0.set_ylim(ax0.get_ylim())
h2 = twax.plot(1e-3*xplot, 1e3 * anomaly(xplot), color=color2, linewidth=1.0, linestyle='--')
l2 = 'Anomaly'
twax.set_ylim(0.0, ax0.get_ylim()[1] - ax0.get_ylim()[0])
ax0.set_xlabel('Lateral distance to fault (km)', labelpad=0.4);
ax0.set_ylabel('Heat flow ($\mathrm{mW\,m}^{-2}$)', labelpad=0.2)
twax.set_ylabel('Anomaly ($\mathrm{mW\,m}^{-2}$)', labelpad=0.2)
ax0.text(-84, 87, "(a)")

ax1 = fig.add_axes((0.435, 0.2, 0.24, 0.77))
h0_1 = ax1.step(np.concatenate(([q.min()], np.sort(q))),
                [0] + list((np.arange(q.size)+1)/q.size), color=color1, linewidth=1.0,
                where='post')
qplot = np.linspace(q.min(), (q+dq).max(), 100)
ax1.plot(qplot, gamma_cdf(qplot, *gamma_mle(q)), linewidth=1.0, linestyle=':',
         color=color1)

h1_1 = ax1.step(np.concatenate(([(q+dq).min()], np.sort(q+dq))),
                [0] + list((np.arange(q.size)+1)/q.size), color=color0, linewidth=0.8,
                where='post')
ax1.plot(qplot, gamma_cdf(qplot, *gamma_mle(q+dq)), linewidth=1.0, linestyle=':',
         color=color0)
ax1.legend([ax1.plot([],[],linewidth=1.0, linestyle=':', color='k')[0]], ['MLE'])

ax1.set_ylim(0,1)
ax1.set_ylabel('CDF', labelpad=0.2)
ax1.set_xlabel('Heat flow ($\mathrm{mW\,m}^{-2}$)', labelpad=0.4)
ax1.text(58, 0.9, "(b)")


ax2 = fig.add_axes((0.755, 0.2, 0.24, 0.77))
h3 = ax2.plot(1e-6*P_H, 1e9*y, color=color0)
l3 = 'Posterior $P_H$\nestimate'
ax2.set_xlim(0.0, 1e-6*P_H.max())
ax2.set_ylim(0.0, 1e9 * 1.03*y.max())
ax2.axvline(1e-6*P_ano, color=color2, linestyle='--', linewidth=1.0)
ax2.set_xlabel('Frictional power $P_H$ (MW)', labelpad=0.4)
ax2.set_ylabel('Posterior density ($\\mathrm{GW}^{-1}$)', labelpad=0.2)
ax2.legend(((h0_0, h0_1[0]), (h1_0, h1_1[0]), h2[0], h3[0]), (l0, l1, l2, l3), fontsize='small',
           title='All panels')
ax2.text(7, 7.8, "(c)")


fig.savefig("figures/A11-Bayesian-Anomaly-Quantification.pdf")

### License
```
A notebook to sketch the d_min permutations and the synthetic fault
heat flow data set configuration.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2022 Malte J. Ziebarth
            

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```